In [1]:
from scipy.ndimage import gaussian_filter1d
import numpy as np
from shapely.geometry import *
import geopandas as gpd
import math

In [2]:
def df_wgs_to_utm(gdf):
    if (gdf.crs["init"]!="epsg:4326"):
        raise Exception("in df_wgs_to_utm - converting GeoDataFrame projection from wgs to utm - input GeoDataFrame is not in wgs. crs is {}".format(gdf.crs))
    points = gdf.geometry.centroid.tolist()
    multi = MultiPoint(points)
    pt = multi.centroid.coords[0]
    epsg = convert_wgs_to_utm(pt[0],pt[1])
    return gdf.to_crs({'init': 'epsg:' + epsg})

def convert_wgs_to_utm(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code


def gaussian_smooth(coords, sigma=2, num_points_factor=2, mode='wrap'):
    
    coords = np.array(coords)
    x, y = coords.T
    xp = np.linspace(0, 1, coords.shape[0])
    interp = np.linspace(0, 1,coords.shape[0] * num_points_factor)
    x = np.interp(interp, xp, x)
    y = np.interp(interp, xp, y)
    x = gaussian_filter1d(x, sigma, mode=mode)
    y = gaussian_filter1d(y, sigma, mode=mode)
    return x, y


def gaussian_smooth_geom(geom, sigma=2, num_points_factor=2, mode='wrap'):


    if isinstance(geom, (Polygon, LineString)):
        if type(geom) == LineString:
            x, y = gaussian_smooth(geom.coords, sigma, num_points_factor, mode)
            return LineString(list(zip(x, y)))

        elif type(geom) == Polygon:
            x, y = gaussian_smooth(geom.exterior.coords,sigma, num_points_factor, mode)
            x = np.append(x, x[0])
            y = np.append(y, y[0])
            if len(list(geom.interiors)) > 0:
                l = []
                l.append(list(zip(x, y)))

                for interior in list(geom.interiors):
                    x, y = gaussian_smooth(interior,sigma, num_points_factor, mode)
                    l.append(list(zip(x, y)))
                return Polygon([item for sublist in l for item in sublist])
            else:

                return Polygon(list(zip(x, y)))
        
    elif isinstance(geom, (MultiPolygon, MultiLineString)):
        list_ = []
        for g in geom:
            list_.append(gaussian_smooth_geom(g, sigma, num_points_factor, mode))

        if type(geom) == MultiPolygon:

            return MultiPolygon(list_)
        else:
            return MultiLineString(list_)
    else:
        warnings.warn('geometry must be LineString, Polygon, MultiLineString or MultiPolygon, returning original geometry')
        return geom
 

In [15]:
df = gpd.GeoDataFrame.from_file('/mnt/storage_4tb/ymi/geo_data/angle_net_data/job_results/1-68/dist_net_resnet_1349_256_aug_lines/detected_lines_no_merge.geojson')
# df = df_wgs_to_utm(df)

In [16]:
df['geometry'] = df['geometry'].apply(lambda x: gaussian_smooth_geom(x,
                                                                      sigma=5,
                                                                      num_points_factor=5,
                                                                      mode = 'nearest'))

In [17]:
df['geometry'] = df['geometry'].apply(lambda x: x.simplify(0.1))

In [18]:
df.to_crs({'init':'epsg:4326'}).to_file('/mnt/storage_4tb/ymi/geo_data/angle_net_data/job_results/1-68/dist_net_resnet_1349_256_aug_lines/detected_lines_no_merge_smooth.geojson',
          driver = 'GeoJSON')